In this notebook, we are going to implement a graph coloring algorithm based on a MIS solver for a practical problem of sharing a bandwidth of frequencies for a network of antennas.

In [ ]:
# imports
from mis.coloring.coloring import GraphColoringSolver
from mis.data.dataloader import DataLoader
from pathlib import Path
from mis.pipeline.config import SolverConfig
from mis.shared.types import MethodType
from mis.pipeline.backends import QutipBackend

## Importing our dataset

The practical dataset of interest is the placement of 5G antennas in Paris that can be found in the `antenna_Paris.csv` file. A set of antennas are distributed over the city with a specific coverage range. Therefore, some antennas will be in range of each other.

In [ ]:
loader = DataLoader()
loader.load_from_csv_coordinates(Path('./datasets/coloring/antenna_Paris.csv'))

## Representing our problem instance
We will represent the problem by a graph, such that each node represents an antenna, with an edge between two if they are in the range of each other. We consider that all the antennas have a constant range R, which we will set to $1.2km$.

In [ ]:
instance = loader.build_mis_instance_from_coordinates(antenna_range=1.2)

Let's visualize our dataset :

In [ ]:
pos = {i: (loader.coordinates_dataset[i][1], loader.coordinates_dataset[i][0]) for i in range(len(loader.coordinates_dataset))}
instance.draw()


## Solving the Graph Coloring Problem

We will use the greedy heuristic algorithm described in [Appendix A](https://arxiv.org/pdf/2301.02637) to find a coloring of the graph using MIS output. 

The algorithm starts with a set $S$ of all the nodes in the graph, and at each iteration it searches for a maximum independent set of nodes of the subgraph formed by the nodes currently in $S$, colors all of the nodes of the MIS in the same color, then removes them from $S$. The operation is repeated until $S$ is empty.


### 1. Using a classical solver

In [ ]:
solver = GraphColoringSolver(loader, 1.2, SolverConfig(
    method = MethodType.EAGER,
    max_iterations=1
))
solver.solve()
solver.visualize_solution()
print(solver.colors)   
print(f"Number of colors used: {solver.colors_count}")

### 2. Using the quantum SDK QuTiP

In [ ]:
solver = GraphColoringSolver(loader, 1.2, SolverConfig(
    method = MethodType.EAGER,
    backend = QutipBackend(),
    max_iterations=1
))
solver.solve()
solver.visualize_solution()
print(solver.colors)   
print(f"Number of colors used: {solver.colors_count}")